<h1 align = "center",style="fontsize:40">CYCLIC REDUCTION ON TRIDIAGONAL SYSTEM</h1>

<h2 style="fontsize:35"><u>Theory</u></h2>

A tridiagonal matrix is commonly encountered in numerical solutions of 1D equations in Physics, such as the time-dependent Schrödinger equation: 

$$
i \hbar \frac{\partial}{\partial t} {\Psi(x,t)} = \left[\frac{-\hbar^2}{2m} \frac{\partial^2}{\partial x^2} + V(x,t) \right] {\Psi(x,t)} 
$$

Discretizing the above equation via the Crank-Nicolson algotrithm leads to a matrix equation
$$
\left(\mathbb{I} + \frac{i \hat{ H } \Delta t}{2} \right) \Psi(x,t + \Delta t)= \left(\mathbb{I} - \frac{i \hat{ H } \Delta t}{2} \right) \Psi(x,t)
$$
where $\mathbb{I}$ is the identitiy operator and $\hat{H} = \hat{T} + \hat{V}$ is the Hamiltonian, with $\hat{V} = \text{diag}(V_i)$ and 

$$
=\frac{-\hbar^2 }{2 m \Delta x^2}
\begin{bmatrix}
-2 & 1  & 0 & 0 &  0 & 0 & ... & 0\\
1  & -2 & 1  & 0 & 0 & 0 & ... & 0\\
0  & 1  & -2 & 1  & 0 & 0 & ... & 0\\
0  & 0  & 1 & -2  & 1 & 0 & ... & 0\\
\vdots   & \vdots  & \vdots & \vdots & \ddots & \vdots & \vdots & \vdots \\
0 & 0 & ... & 0 & 1 & -2 & 1 & 0\\
0 & 0 & ... & 0 & 0 & 1 & -2 & 1\\
0 & 0 & ... & 0 & 0 & 0 & 1 & -2\\
\end{bmatrix}
$$

This is a system of linear equations where $\Psi(x,t+\Delta t)$ is a vector of unknowns, and $\hat{H}$ is a tridiagonal matrix. There are several approaches to solve such a system, but I will apply the Cyclic Reduction algorithm in this notebook to obtain an exact solution. This method was designed for systems of the form $A.\bf{x} = {d}$, where $\bf{x}$ is a vector of unknowns, $\bf{d}$ is a vector of known constants, and 
$$
A =
\begin{pmatrix}
b_1 & c_1 & 0      & \cdots & 0      \\
a_2 & b_2 & c_2    & \ddots & \vdots \\
0   & a_2 & b_3    & \ddots & 0      \\
\vdots& \ddots& \ddots & \ddots & c_{N-1} \\
0   & \cdots& 0      & a_{N} & b_N
\end{pmatrix}
$$

is a matrix of size $2^n\times2^n$, although generalizations are possible. The idea is to simplify the system in $n$ steps by eliminating half of the equations at every step until two equations remain. These are then directly solved, with the rest of the unknowns  obtained by successively substituting the elements already calculated. 



<h3>Example: $n = 3$</h3>

As an example, consider the following $8 \times 8$ system:

$$
\begin{pmatrix}
b_1 & c_1 & 0   & 0   & 0   & 0   & 0   & 0   \\
a_2 & b_2 & c_2 & 0   & 0   & 0   & 0   & 0   \\
0   & a_3 & b_3 & c_3 & 0   & 0   & 0   & 0   \\
0   & 0   & a_4 & b_4 & c_4 & 0   & 0   & 0   \\
0   & 0   & 0   & a_5 & b_5 & c_5 & 0   & 0   \\
0   & 0   & 0   & 0   & a_6 & b_6 & c_6 & 0   \\
0   & 0   & 0   & 0   & 0   & a_7 & b_7 & c_7 \\
0   & 0   & 0   & 0   & 0   & 0   & a_8 & b_8
\end{pmatrix}
\begin{pmatrix}
x_1 \\
x_2 \\
x_3 \\
x_4 \\
x_5 \\
x_6 \\
x_7 \\
x_8
\end{pmatrix}
=
\begin{pmatrix}
d_1 \\
d_2 \\
d_3 \\
d_4 \\
d_5 \\
d_6 \\
d_7 \\
d_8
\end{pmatrix}
$$
which represents a set of eight equations:
$$
\begin{align*}
b_1 x_1 + c_1 x_2 &= d_1 \\
a_2 x_1 + b_2 x_2 + c_2 x_3 &= d_2 \\
a_3 x_2 + b_3 x_3 + c_3 x_4 &= d_3 \\
a_4 x_3 + b_4 x_4 + c_4 x_5 &= d_4 \\
a_5 x_4 + b_5 x_5 + c_5 x_6 &= d_5 \\
a_6 x_5 + b_6 x_6 + c_6 x_7 &= d_6 \\
a_7 x_6 + b_7 x_7 + c_7 x_8 &= d_7 \\
a_8 x_7 + b_8 x_8 &= d_8
\end{align*}
$$

We begin by isolating the the odd-indexed unknowns $x_i (i=1,3,5,7)$ in the odd-indexed equations:
$$
\begin{align*}
x_1 &= \frac{d_1 - c_1 x_2}{b_1} \\
x_3 &= \frac{d_3 - a_3 x_2 - c_3 x_4}{b_3} \\
x_5 &= \frac{d_5 - a_5 x_4 - c_5 x_6}{b_5} \\
x_7 &= \frac{d_7 - a_7 x_6 - c_7 x_8}{b_7}
\end{align*}
$$
and then substituing these values back into the even-indexed equations, which yields 4 equations for the even-indexed unknowns
$$
\begin{align*}
x_2 \left( b_2 - \frac{a_2 c_1}{b_1} - \frac{c_2 a_3}{b_3} \right) + x_4 \left( -\frac{c_2 c_3}{b_3} \right) &= d_2 - \frac{a_2 d_1}{b_1} - \frac{c_2 d_3}{b_3} \\
x_2 \left( -\frac{a_4 a_3}{b_3} \right) + x_4 \left( b_4 - \frac{a_4 c_3}{b_3} - \frac{c_4 a_5}{b_5} \right) + x_6 \left( -\frac{c_4 c_5}{b_5} \right) &= d_4 - \frac{a_4 d_3}{b_3} - \frac{c_4 d_5}{b_5} \\
x_4 \left( -\frac{a_6 a_5}{b_5} \right) + x_6 \left( b_6 - \frac{a_6 c_5}{b_5} - \frac{c_6 a_7}{b_7} \right) + x_8 \left( -\frac{c_6 c_7}{b_7} \right) &= d_6 - \frac{a_6 d_5}{b_5} - \frac{c_6 d_7}{b_7} \\
x_6 \left( -\frac{a_8 a_7}{b_7} \right) + x_8 \left( b_8 - \frac{a_8 c_7}{b_7} \right) &= d_8 - \frac{a_8 d_7}{b_7}
\end{align*}
$$

Re-labelling the constants, 
$$
\begin{align*}
b_2' x_2 + c_2' x_4  &= d_2' \\
a_4' x_2  + b_4' x_4 + c_4' x_6  &= d_4' \\
a_6' x_4  + b_6' x_6 + c_6' x_8 &= d_6'\\
a_8' x_6  + b_8' x_8 &= d_8'
\end{align*}
$$
and repeating the same procedure as before results in a pair of equations:
$$
\begin{align*}
b_4''  x_4 + c_4'' x_8  &= d_4' \\
a_4'' x_4  + b_8'' x_8 &= d_8'
\end{align*}
$$
This brings us to the end of the Forward Reduction procedure, where the coeffecient update at step $s$(=$1,2$ in this example) on every even equation $a_i x_{i-1} + b_i x_i + c_{i-1}x_{i+1} = d_i $ followed the formula:
$$
\begin{align*}
a_i^{(s)} &= k_1^{(s)} c_{i-\Delta}^{(s-1)} \\
b_i^{(s)} &= b_i^{(s-1)} + k_1^{(s)} a_{i-\Delta}^{(s-1)} + k_2^{(s)} c_{i+\Delta}^{(s-1)} \\
c_i^{(s)} &= k_2^{(s)} a_{i+\Delta}^{(s-1)} \\
d_i^{(s)} &= d_i^{(s-1)} + k_1^{(s)} d_{i-\Delta}^{(s-1)} + k_2^{(s)} d_{i+\Delta}^{(s-1)}
\end{align*}
$$
where 
$$
k_1^{(s)} = -\frac{a_i^{(s-1)}}{b_{i-\Delta}^{(s-1)}}, \quad k_2^{(s)} = -\frac{c_i^{(s-1)}}{b_{i+\Delta}^{(s-1)}}
$$
and $\Delta = 2^{s-1}$.

Solving the final two equations provides the unknowns $x_4$ and $x_8$($x_{N/2}$ and $x_N$ in the general case). No further modification of coefficients is necessary, since these two values can now be substituted into the previous set of equations to obtain $x_2$ and $x_6$, and so on. This is known as the Backward Substitution phase of the algorithm, which follows following formula at step $s$:
$$
x_i^{(s)} = \frac{d_i^{(n-s)} - a_i^{(n-s)} x_{i-\Delta}^{(n-1)} - c_i^{(n-s)} x_{i+\Delta}^{(s-1)}}{b_i^{(n-s)}}
$$
The final step of the Backward phase ensures that all odd-indexed values of $\bf{x}$ are filled out as well, since $\Delta$ becomes $1$ in at that point.

<h2><h2 style="fontsize:35"><u>Python Code</u></h2>
This method can be implemented on Python by iteratively updating the coefficient vectors $\bf{a},\bf{b},\bf{c}$ and $\bf{d}$ according to the formulae listed above. Care must be taken to account for the fact that counter variables and array indexing both begin at $0$ rather than 1. The following function reads in 4 vectors $\bf{a},\bf{b},\bf{c}$ and $\bf{d}$, and returns the vector $\bf{x}$ calculated via the Cyclic Reduction algorithm. 

In [1]:
import numpy as np
import time as time 
from scipy.linalg import solve_banded

def Cyclic_Reduction(a, b, c, d, N, n):

    "-------------------------------------------FORWARD REDUCTION-------------------------------------------"
    for step in range(n-1):                                          #do this for all n steps
        for iEven in range(2**(step + 1) - 1, N, 2**(step + 1)):     #loop through every even equation in a step
            Delta = 2**step
            iLeft = iEven - Delta
            iRight = iEven + Delta

            k1 = a[iEven] / b[iLeft]
            #deal with boundary
            if(iRight>=N):
                iRight = 0      #some dummy index
                k2 = 0
            else:
                k2 = c[iEven] / b[iRight]

            b[iEven] = b[iEven] - k1*c[iLeft] - k2*a[iRight] 
            d[iEven] = d[iEven] - k1*d[iLeft] - k2*d[iRight] 
            a[iEven] = - k1* a[iLeft] 
            c[iEven] = - k2 *c[iRight] 
 
    "-----------------------------------------SOLVE FINAL 2 EQUATIONS---------------------------------------"
    #initialize unknown array
    x = np.zeros_like(a,dtype = float)
    iMiddle = int(N/2)-1;
    iFinal = (N-1)
    x[iMiddle] = (d[iMiddle] - c[iMiddle]*d[iFinal]/b[iFinal])/(b[iMiddle]-c[iMiddle]*a[iFinal]/b[iFinal])
    x[iFinal] = (d[iFinal]-a[iFinal]*d[iMiddle]/b[iMiddle])/(b[iFinal]-a[iFinal]*c[iMiddle]/b[iMiddle])

    "------------------------------------------BACKWARD SUBSTITUTION----------------------------------------"
    for step in range(n-2, -1, -1):                                   #do this for all n steps
        for iKnown in range(2 ** (step + 1) - 1, N, 2 ** (step + 1)): #loop through every solved equation in a step
            Delta = 2** step
            iLeft = iKnown - Delta
            iRight = iKnown + Delta
            #exit loop upon hitting the boundary
            if iRight >= N:
                break
        
            x[iLeft] = (d[iLeft] - a[iLeft] * x[iLeft - Delta] - c[iLeft] * x[iLeft + Delta]) / b[iLeft]
            x[iRight] = (d[iRight] - a[iRight] * x[iRight - Delta] - c[iRight] * x[iRight + Delta]) / b[iRight]

    return x


We will also calculate the value according to $\texttt{np.linalg.solve}$ and compare the results and duration of the two methods.

In [2]:
if __name__ == "__main__":
    
    n = 10
    N = 2**n
    
    #set values for diagonals
    a = np.full(N,1.0)
    b = np.full(N,-2.0)
    c = np.full(N,1.0)
    d = np.full(N,0.1)
    
    #store LHS matrix and RHS vector for later use
    A = np.diag(a[1:],-1) + np.diag(b) + np.diag(c[:-1],1)
    d0 = np.copy(d)
    a[0] = 0.
    c[-1] = 0.
    #numpy via numpy
    start1 = time.time()
    x_numpy = np.linalg.solve(A,d)
    stop1 = time.time()

    #apply CR
    start2 = time.time()
    x_CR = Cyclic_Reduction(a, b, c, d, N, n)
    stop2 = time.time()
    
    print("Numpy time =",stop1-start1)
    print("CR time =",stop2-start2)
    
    error_CR = np.dot(A,x_CR) - d0
    error_numpy = np.dot(A,x_numpy) - d0
    print("\nNumpy Relative error(Norm)= ",np.linalg.norm(error_numpy)/np.linalg.norm(x_numpy))
    print("CR Relative error(Norm)= ",np.linalg.norm(error_CR)/np.linalg.norm(x_CR))


Numpy time = 0.01876688003540039
CR time = 0.0016069412231445312

Numpy Relative error(Norm)=  1.3357493260017978e-16
CR Relative error(Norm)=  1.4286856643536398e-16


Our code is significantly faster than $\texttt{numpy.linalg.solve}$ while being just as accurate. Admittedly, the latter is a general purpose solver, so let's compare our code against $\texttt{scipy.solve\_banded}$, which specializes in solving banded matrices. 

In [3]:
if __name__ == "__main__":
    
    n = 10
    N = 2**n
    
    #set values for diagonals
    a = np.full(N,1.0)
    b = np.full(N,-2.0)
    c = np.full(N,1.0)
    d = np.full(N,0.1)

    #store LHS matrix and RHS vector for error computation
    A = np.diag(a[1:],-1) + np.diag(b) + np.diag(c[:-1],1)
    d0 = np.copy(d)
    
    a[0] = 0.
    c[-1] = 0.
    start1 = time.time()
    ab = np.vstack((a,b,c))               #stack diags into a 3x1 matrix
    x_scipy = solve_banded((1, 1), ab, d) 
    stop1 = time.time()
    
    #apply CR
    start2 = time.time()
    x_CR = Cyclic_Reduction(a, b, c, d, N, n)
    stop2 = time.time()
    
    print("Scipy time =",stop1-start1)
    print("CR time =",stop2-start2)
    
    error_CR = np.dot(A,x_CR) - d0
    error_scipy = np.dot(A,x_scipy) - d0
    print("\nScipy Relative error(Norm)= ",np.linalg.norm(error_scipy)/np.linalg.norm(x_scipy))
    print("CR Relative error(Norm)= ",np.linalg.norm(error_CR)/np.linalg.norm(x_CR))

Scipy time = 0.00014209747314453125
CR time = 0.0014121532440185547

Scipy Relative error(Norm)=  1.3357493260017978e-16
CR Relative error(Norm)=  1.4286856643536398e-16


Here we see that the Cyclic Reduction method is significantly slower, but that is down to the serial nature of our implementation. We can take advantage of parallel computing to significantly speed up the calculations involved and comprehensively beat $\texttt{scipy}$. See the other folder in this repository.